# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,0.68,85,100,5.14,US,1704593658
1,1,puerto ayora,-0.7393,-90.3518,25.31,95,88,2.24,EC,1704593635
2,2,port-aux-francais,-49.3500,70.2167,5.23,82,99,10.87,TF,1704593776
3,3,zapata,26.9073,-99.2714,15.06,51,0,0.00,US,1704593776
4,4,palikir - national government center,6.9248,158.1611,30.47,55,20,6.69,FM,1704593721


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
every_city = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)
    

# Display the map
every_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp = city_data_df.loc[city_data_df['Max Temp'] < 27, :]
max_temp_final = max_temp.loc[max_temp['Max Temp'] > 21, :]
wind_speed = max_temp_final.loc[max_temp_final['Wind Speed'] < 4.5, :]
cloudiness = wind_speed.loc[wind_speed['Cloudiness'] == 0, :]
cloudiness

# Drop any rows with null values
city_data_clean = cloudiness.dropna()

# Display sample data
city_data_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
123,123,ormara,25.2088,64.6357,21.32,58,0,3.02,PK,1704593929
134,134,santo antonio,-6.3106,-35.4789,24.54,83,0,2.04,BR,1704593931
190,190,bansang,13.4333,-14.6500,24.29,15,0,2.82,GM,1704593948
235,235,salalah,17.0151,54.0924,22.05,37,0,3.09,OM,1704593963
266,266,coxen hole,16.3244,-86.5366,23.60,88,0,1.54,HN,1704593971
299,299,san pedro pochutla,15.7333,-96.4667,22.54,80,0,1.10,MX,1704593978
300,300,centenario,-38.8043,-68.1400,25.86,31,0,3.09,AR,1704593979
339,339,nova sintra,14.8667,-24.7167,21.52,69,0,1.63,CV,1704593724
380,380,cova figueira,14.8905,-24.2934,21.44,70,0,3.60,CV,1704593996
513,513,celestun,20.8667,-90.4000,23.49,91,0,3.90,MX,1704594048


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_clean.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
123,ormara,PK,25.2088,64.6357,58,
134,santo antonio,BR,-6.3106,-35.4789,83,
190,bansang,GM,13.4333,-14.6500,15,
235,salalah,OM,17.0151,54.0924,37,
266,coxen hole,HN,16.3244,-86.5366,88,
299,san pedro pochutla,MX,15.7333,-96.4667,80,
300,centenario,AR,-38.8043,-68.1400,31,
339,nova sintra,CV,14.8667,-24.7167,69,
380,cova figueira,CV,14.8905,-24.2934,70,
513,celestun,MX,20.8667,-90.4000,91,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ormara - nearest hotel: No hotel found
santo antonio - nearest hotel: Pousada Onça Puma
bansang - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
coxen hole - nearest hotel: Elizabeth Inn
san pedro pochutla - nearest hotel: Olimpia
centenario - nearest hotel: Hotel Global
nova sintra - nearest hotel: Residência Ka Dencho
cova figueira - nearest hotel: No hotel found
celestun - nearest hotel: Hotel Flamingo Playa


,City,Country,Lat,Lng,Humidity,Hotel Name
123,ormara,PK,25.2088,64.6357,58,No hotel found
134,santo antonio,BR,-6.3106,-35.4789,83,Pousada Onça Puma
190,bansang,GM,13.4333,-14.6500,15,No hotel found
235,salalah,OM,17.0151,54.0924,37,Muscat International Hotel
266,coxen hole,HN,16.3244,-86.5366,88,Elizabeth Inn
299,san pedro pochutla,MX,15.7333,-96.4667,80,Olimpia
300,centenario,AR,-38.8043,-68.1400,31,Hotel Global
339,nova sintra,CV,14.8667,-24.7167,69,Residência Ka Dencho
380,cova figueira,CV,14.8905,-24.2934,70,No hotel found
513,celestun,MX,20.8667,-90.4000,91,Hotel Flamingo Playa


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
every_city = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
every_city

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)